# Day Activities

## Problem 1: Exercise problems

In [63]:
import numpy as np

LIKELIHOOD = np.array([
                        [0.25],
                        [1],
                       ])

def bayes_filter(prior: np.ndarray):
    numerator = LIKELIHOOD * prior
    normalizer = np.sum(numerator)
    return numerator / normalizer

# Bayes loop from N = 1,2,...,10

# Initialize prior = [nf ; f] <- column vector
prior = np.array([
                [0.985],
                [0.015]
                ])

for i in range(10):
    print(f"Posterior t={i+1}:")
    pxz = bayes_filter(prior)
    print(pxz)
    # update prior
    prior = pxz

Posterior t=1:
[[0.94258373]
 [0.05741627]]
Posterior t=2:
[[0.80408163]
 [0.19591837]]
Posterior t=3:
[[0.50642674]
 [0.49357326]]
Posterior t=4:
[[0.20414508]
 [0.79585492]]
Posterior t=5:
[[0.06026308]
 [0.93973692]]
Posterior t=6:
[[0.01577893]
 [0.98422107]]
Posterior t=7:
[[0.00399198]
 [0.99600802]]
Posterior t=8:
[[0.00100099]
 [0.99899901]]
Posterior t=9:
[[2.50435720e-04]
 [9.99749564e-01]]
Posterior t=10:
[[6.26206918e-05]
 [9.99937379e-01]]


### Exercise 2: Robot Tag

In [64]:
from enum import Enum

TRANSITION_MAT = np.array([[0, 0, 0],
                           [0.5, 0.8, 0.3],
                           [0.5, 0.2, 0.7]])

SENSOR_MAT = np.array([[0, 0.5, 0.5],
                       [0, 0.9, 0.1],
                       [0, 0.1, 0.9]])

class State(Enum):
    ROOM1 = 0
    ROOM2 = 1
    ROOM3 = 2

### Part 1
In one version of the game, the tagged robot shuts down for a few seconds to give the other robot a chance to run away. Our “it” robot just wakes up after some set time, and would like to estimate where the other robot is in the world. After one world timestep, what is the probability distribution over where the other robot is? Over two timesteps? Continue computing a prediction further into the future – what do you notice about the distribution?

In [65]:
prior = np.array([[1.0, 0.0, 0.0]])
print(f"Prediction at t = 0: {prior =}")
for i in range(30):
    prior@=TRANSITION_MAT.T
    print(f"Prediction at t = {i+1}: {prior =}")

Prediction at t = 0: prior =array([[1., 0., 0.]])
Prediction at t = 1: prior =array([[0. , 0.5, 0.5]])
Prediction at t = 2: prior =array([[0.  , 0.55, 0.45]])
Prediction at t = 3: prior =array([[0.   , 0.575, 0.425]])
Prediction at t = 4: prior =array([[0.    , 0.5875, 0.4125]])
Prediction at t = 5: prior =array([[0.     , 0.59375, 0.40625]])
Prediction at t = 6: prior =array([[0.      , 0.596875, 0.403125]])
Prediction at t = 7: prior =array([[0.       , 0.5984375, 0.4015625]])
Prediction at t = 8: prior =array([[0.        , 0.59921875, 0.40078125]])
Prediction at t = 9: prior =array([[0.        , 0.59960938, 0.40039062]])
Prediction at t = 10: prior =array([[0.        , 0.59980469, 0.40019531]])
Prediction at t = 11: prior =array([[0.        , 0.59990234, 0.40009766]])
Prediction at t = 12: prior =array([[0.        , 0.59995117, 0.40004883]])
Prediction at t = 13: prior =array([[0.        , 0.59997559, 0.40002441]])
Prediction at t = 14: prior =array([[0.        , 0.59998779, 0.40001

We see that the prediction converges to [0, 0.6, 0.4] starting from t = 26. Since we have no observations, the distribution converges to the bias present in the transition matrix.

### Part 2
In another version of the game, there is no shutdown period, but our “it” robot can only take noisy measurements (model in the table below) of where the other robot is according to a measurement model. While our “it” robot is still certain that the other robot started in Room 1, over the next 5 timesteps it observes the other robot in {Room 2, Room 3, Room 3, Room 2, Room 3}. Using filtering, what is the point-wise most likely trajectory of the chased robot? Using smoothing, what is the most likely trajectory of the chased robot?

In [66]:
def forward_step(alpha_k: np.ndarray, Mmat: np.ndarray, Tmat: np.ndarray, z_kplus1: State):
    """
    Referenced from implementation example

    alpha_k: forward step result from prior step; 1x3 row vector
    Mmat: measurement model matrix; 3x3
    Tmat: transition model matrix; 3x3
    z_kplus1: new observation about robot state; State Enum
    """
    alpha_temp = np.zeros_like(alpha_k)
    # Sum up all possible transition from previous state; alpha_k * T_qs
    for room in State:
        alpha_temp += alpha_k[room.value] * Tmat[:,room.value]
    return Mmat[:, z_kplus1.value] * alpha_temp

initial_state = np.array([1, 0, 0])  # initial state of the world
observations = np.array([State.ROOM3, State.ROOM3, State.ROOM2, State.ROOM3])  # history of observations; 0 -> room 1, 1 -> room 2, 2 -> room 3
steps = 4

alpha = initial_state * SENSOR_MAT[:, State.ROOM2.value]
print("--- FORWARD STEP ---")
print(f"Forward step k = 1: {alpha}")
forward_pass = [alpha]

for i, z in enumerate(observations):
    alpha = forward_step(alpha, SENSOR_MAT, TRANSITION_MAT, z)
    print(f"Forward step k = {i + 2}: {alpha}")
    forward_pass.append(alpha)

print("\n--- FILTERED STEP ---")
for i, a in enumerate(forward_pass):
    filtered = a / np.sum(a)
    print(f"Filtered estimate k = {i + 1}: {filtered}")


--- FORWARD STEP ---
Forward step k = 1: [0.5 0.  0. ]
Forward step k = 2: [0.    0.025 0.225]
Forward step k = 3: [0.      0.00875 0.14625]
Forward step k = 4: [0.        0.0457875 0.0104125]
Forward step k = 5: [0.         0.00397538 0.01480163]

--- FILTERED STEP ---
Filtered estimate k = 1: [1. 0. 0.]
Filtered estimate k = 2: [0.  0.1 0.9]
Filtered estimate k = 3: [0.         0.05645161 0.94354839]
Filtered estimate k = 4: [0.        0.8147242 0.1852758]
Filtered estimate k = 5: [0.         0.21171513 0.78828487]


In [67]:
def backward_step(B_kplus1: np.ndarray, Mmat: np.ndarray, Tmat: np.ndarray, z_kplus1: State):
    B_k = np.zeros_like(B_kplus1)
    for s in State:
        # compute B * P(x_k = s | x_k+1 = q) * P(z_k+1 | x_k+1 = q)
        # compute for all q at once
        B_k[s.value] += np.sum(B_kplus1 * Tmat[:, s.value] * Mmat[:, z_kplus1.value])
    return B_k

B_last = np.array([1.0, 1.0, 1.0])
observations = np.array([State.ROOM2, State.ROOM3, State.ROOM3, State.ROOM2, State.ROOM3])
print(f"--- BACKWARD STEP ---")
print(f"Backward step k = 5: {B_last}")
backward_pass = [B_last]

for i in range(4,0,-1): # iterate backwards
    B_last = backward_step(B_last, SENSOR_MAT, TRANSITION_MAT, observations[i])
    print(f"Backward step k = {i}: {B_last}")
    backward_pass.append(B_last)

# Smoothing step with forward and backward
print(f"\n--- SMOOTHING STEP ---")
for i, (fwd, bwd) in enumerate(zip(forward_pass, backward_pass[::-1])):
    numerator = fwd * bwd
    print(f"Smoothing step k = {i}: {numerator/sum(numerator)}")


--- BACKWARD STEP ---
Backward step k = 5: [1. 1. 1.]
Backward step k = 4: [0.5  0.26 0.66]
Backward step k = 3: [0.15   0.2004 0.1164]
Backward step k = 2: [0.0624   0.036984 0.079344]
Backward step k = 1: [0.037554   0.01724064 0.05109624]

--- SMOOTHING STEP ---
Smoothing step k = 0: [1. 0. 0.]
Smoothing step k = 1: [0.         0.04924109 0.95075891]
Smoothing step k = 2: [0.         0.09338552 0.90661448]
Smoothing step k = 3: [0.         0.63400703 0.36599297]
Smoothing step k = 4: [0.         0.21171513 0.78828487]
